In [9]:
import requests
from pprint import pprint
import re
import os
import pandas as pd
import json


In [10]:
# get info about catalog-record (Lake/Reservoir level data)
catalog_record_id = 2304
headers = {"accept": "application/vnd.api+json"}
url = f"https://data.usbr.gov/rise/api/catalog-record/{catalog_record_id}"
resp = requests.get(url, headers=headers)

In [11]:
resp_dict = resp.json()
pprint(resp_dict)

{'data': {'attributes': {'_id': 2304,
                         'catalogStatusId': 1,
                         'createDate': '2021-02-05T10:17:48+00:00',
                         'generationEffort': 'California-Great Basin Water '
                                             'Operations Monitoring',
                         'generationEffortId': 52,
                         'metadataFilePath': 'rise/catalog-record/metadata/metadata_rise_record_2304.xml',
                         'metadataStandard': {'_id': 'ISO',
                                              'createDate': '2018-11-20T13:25:52+00:00',
                                              'definition': 'International '
                                                            'Standards '
                                                            'Organization '
                                                            '(ISO) '
                                                            '19115-1:2014 '
                      

In [12]:
catalog_items = resp_dict['data']['relationships']['catalogItems']['data']  # get the catalog items associated with this 
catalog_items[:3]

[{'type': 'CatalogItem', 'id': '/rise/api/catalog-item/10772'},
 {'type': 'CatalogItem', 'id': '/rise/api/catalog-item/10773'},
 {'type': 'CatalogItem', 'id': '/rise/api/catalog-item/10774'}]

In [7]:
# read in the list of catalog records

from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_catalog_item(catalog_item):
    try:
        headers = {"accept": "application/vnd.api+json"}
        url = f"https://data.usbr.gov{catalog_item['id']}"
        # print(url)
        catalog_item_resp = requests.get(url, headers=headers)

        # print(' '.join([c.strip("'") for c in catalog_item_resp.json()['data']['attributes']['itemTitle'].split('\n')]))
        catalog_item_id = catalog_item_resp.json()['data']['attributes']['_id']
        catalog_record_id = int(catalog_item_resp.json()['data']['relationships']['catalogRecord']['data']['id'].split('/')[-1])

        url_ts = f"https://data.usbr.gov/rise/api/result/download?type=json&itemId={catalog_item_id}&before=2022-10-30&after=1970-01-01&order=ASC"
        ts_resp = requests.get(url_ts)

        savefn = f"../Data/rest/{catalog_record_id}/{catalog_item_id}.json"
        if not os.path.isdir(os.path.split(savefn)[0]):
            os.makedirs(os.path.split(savefn)[0])
        with open(savefn, 'w') as f:
            f.write(ts_resp.text)

        data_df = pd.read_json(savefn)
        data_df.T.to_csv(savefn.replace('.json', '.csv'))
    except Exception as e:
        print(e)

def run_tqdm_multithreaded(f, my_iter):
    l = len(my_iter)
    with tqdm(total=l) as pbar:
        # let's give it some more threads:
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(f, arg): arg for arg in my_iter}
            results = {}
            for future in as_completed(futures):
                arg = futures[future]
                results[arg['id']] = future.result()
                pbar.update(1)

catalogIDs = pd.read_csv("../RISE-records.csv")
for catalogID in tqdm(catalogIDs['catalogID']):
    try:
        headers = {"accept": "application/vnd.api+json"}
        url = f"https://data.usbr.gov/rise/api/catalog-record/{catalogID}"
        resp = requests.get(url, headers=headers)
        resp_dict = resp.json()
        catalog_items = resp_dict['data']['relationships']['catalogItems']['data']
        run_tqdm_multithreaded(download_catalog_item, catalog_items)
        # for catalog_item in tqdm(catalog_items):
        #     download_catalog_item(catalog_item)
    except Exception as e:
        print(e)


  0%|          | 0/166 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]